In [1]:
import argparse
import glob

import seaborn as sns
import numpy as np
import nibabel as nib
from nilearn import datasets
from statsmodels.stats.multitest import multipletests
from src.custom_plotting import plot_surface_stats, mkNifti
from nilearn import surface
from pathlib import Path

In [2]:
def load_noise_ceiling(mask_dir, sid, mask):
    noise_ceiling = np.load(f'{mask_dir}/sub-{sid}_set-test_stat-rho_statmap.npy')
    noise_ceiling = noise_ceiling[mask]
    noise_ceiling[noise_ceiling < 0.] = np.nan
    return noise_ceiling

In [3]:
def filter_r(rs, ps, p_crit=0.05, correct=True, threshold=True):
    if correct:
        _, ps_corrected, _, _ = multipletests(ps, method='fdr_bh')
    else:
        ps_corrected = ps.copy()

    if threshold:
        rs[ps_corrected >= p_crit] = 0.
    else:
        rs[rs < 0.] = 0.
    return rs, ps_corrected

In [4]:
def plot(r_, p_, sid, mask_dir, mask, img, fsaverage):
    p_ -= 1/5001
    r, _ = filter_r(r_, p_)
    # noise_ceiling = load_noise_ceiling(mask_dir, sid, mask)
    # r_normed = np.divide(r_, noise_ceiling)
    r_normed = r.copy()
    volume = mkNifti(r_normed, mask, img)
    texture = {'left': surface.vol_to_surf(volume, fsaverage['pial_left']),
               'right': surface.vol_to_surf(volume, fsaverage['pial_right'])}

    plot_surface_stats(fsaverage, texture,
                       roi=roi_parcel,
                       cmap=cmap,
                       modes=['lateral', 'ventral'],
                       threshold=0.01,
                       # vmax=1.,
                       output_file=f'{figure_dir}/sub-{sid}.png')

In [5]:
data_dir='/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/raw'
out_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim'
mask_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim/Reliability'
top_fig_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/reports/figures/PlotVoxelEncodingTest'

fsaverage = datasets.fetch_surf_fsaverage()
cmap = sns.color_palette('magma', as_cmap=True)
models = ['visual', 'socialprimitive', 'social']
roi_parcel = None
img = nib.load(f'{mask_dir}/sub-01_set-test_stat-rho_statmap.nii.gz')
n_perm = 5000
n_subjs = 3

In [6]:
for model in models:
    # subs = np.zeros((np.prod(img.shape)))
    # null = np.zeros((np.prod(img.shape), n_perm))
    # masks = np.zeros((np.prod(img.shape)))
    for sid_ in range(n_subjs):
        sid = str(sid_+1).zfill(2)
        figure_dir = f'{top_fig_dir}/{model}'
        Path(figure_dir).mkdir(parents=True, exist_ok=True)

        r_true = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-{model}_method-test_rs.npy')
        # r_null = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-{model}_method-test_rnull.npy')
        p = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-{model}_method-test_ps.npy')
        mask = np.load(f'{out_dir}/Reliability/sub-all_set-test_reliability-mask.npy').astype('bool')
        plot(r_true, p, sid, mask_dir, mask, img, fsaverage)

        # subs += mkNifti(r_true, mask, img, nii=False)
        # null += mkNifti(r_null, mask, img, nii=False)
        # masks += mask.astype('float')


In [7]:
# subs /= n_subjs
# null /= n_subjs
# masks /= n_subjs
#
# plot(subs, p, 'all', mask_dir, mask, img, fsaverage)